In [1]:
import pyodbc
from piLang.piLang.SQLTools import SQLTools
from prettytable import PrettyTable
from piLang.piLang.BoxPlot import BoxPlot
from piLang.piLang.LangValidator import LangValidator, PatternFormat
from validator.MotherCodeValidator import MotherCodeValidator
import time

"""
This code will extract and validate the Mother code record in accordance with the QH PDC file format spec.

"""
cnxn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server}; server=localhost\SQLEXPRESS; database=MaterDB; uid=Shane; password=; trusted_connection=Yes')
cursor = cnxn.cursor()
cursor.execute('SELECT  * FROM dbo.[Mothers Code Record] ORDER BY [Mothers UR number]') 
t = SQLTools(cursor)
cnxn.close()
cnxn = None

print(t)
mother_code_meta = { 
    'Transaction Type':    {'Size': 1,   'Type': "char",     'Mandatory': True,  'Enum': ['N','A','D']},
    'Mothers UR number':   {'Size': 12,  'Type': "string",   'Mandatory': True,  'Composite': ['%1','facility_id', 'Code Type']},
    'Date of confinement': {'Size': 10,  'Type': "date",     'Mandatory': True,  'Format': PatternFormat.DATE_YYMMDD.value},
    'Code Type':           {'Size': 1,   'Type': "char",     'Mandatory': True,  'Enum':['C', 'T', 'M', 'P', 'O', 'L', 'A', 'E']},
    'Mothers code':        {'Size': 120,   'Type': "string",   'Mandatory': False, 'AllowBlank': True},
    'facility_id':         {'Mandatory': True}
}
# validate the field length

print("\n---> Running langvalidator ...")
stime = time.time()
lang_validator = LangValidator(t.rs, mother_code_meta)
lang_validator.validate()
lang_validator.counters.calcMean()
lang_validator.counters.calcPercent()
print("Completed in " + str(time.time() - stime) + " secs")
print(lang_validator.getErrors())
print(lang_validator.counters)


print("\n---> Running PDCValidator ...")
stime = time.time()
pdc_validator = MotherCodeValidator(t.rs, mother_code_meta)
pdc_validator.validate()
pdc_validator.counters.calcMean()
pdc_validator.counters.calcPercent()
print("Completed in " + str(time.time() - stime) + " secs")
print(pdc_validator.getErrors())
print(pdc_validator.counters)

counters = list()
counters.extend(lang_validator.counters.toList())
counters.extend(pdc_validator.counters.toList())

#print("\n--> Plot:")

x = BoxPlot()
x.plot(counters)

print("\n\n### DONE ###")



+------------------+-------------------+---------------------+-----------+-------------------------------------------------------------------------------------------------------+-------------+--------------------------------------+
| Transaction Type | Mothers UR number | Date of confinement | Code Type |                                              Mothers code                                             | facility_id |               load_id                |
+------------------+-------------------+---------------------+-----------+-------------------------------------------------------------------------------------------------------+-------------+--------------------------------------+
|        N         |       (Null)      |      24/01/2019     |     A     |                                                   03                                                  |    370.0    | 5569B7B7-104A-4E98-9789-B693332433D7 |
|        N         |     6297266.0     |      28/01/2019     |     L    

Completed in 6.1475841999053955 secs
Error: Mandatory field 'Mothers UR number' is BLANK or NULL. A value is required.
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6297405.0318.0O'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6298725.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6299055.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6300723.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6300723.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6301587.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6302157.0318.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6302688.03.0A'
Error: Duplicate composite key: 'Mothers UR number+facility_id+Code Type', value: '6302756.03



### DONE ###
